In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
DATA_PATH = "../../data/"
CONSULTING_PATH = "Law_Consulting_Data.csv"

consulting = pd.read_csv(DATA_PATH + CONSULTING_PATH, index_col=0)

# 상담 데이터 처리

1. 답변에서 마지막 필요 없는 주의 문구 제거
2. 답변에서 괄호 추출
3. 필요있는 괄호만 추출
    * 법령
    * 판결문
4. 괄호 없는 답변 제거

In [3]:
consulting_df = consulting.copy()
consulting_df.head()

,구분,제목,질문,답변
0,노동 > 근로일반,회사분할로 퇴직금 일괄수령 후 재입사 형식 취한 경우 계속근로인지,저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서...,기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하...
1,노동 > 근로일반,학습지판매위탁 상담교사도 근로기준법상의 근로자인지,저는 학습지제작·판매회사인 甲회사와 위탁업무계약을 체결한 교육상담교사입니다. 저와 ...,「근로기준법」상 근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 ...
2,노동 > 근로일반,대학병원의 전공의가 근로기준법상의 근로자에 해당하는지,저는 甲대학병원에서 전공의 과정을 밟고 있는 레지던트입니다. 그런데 몇 달 전부터 ...,「근로기준법」제2조 제1항 1호에서는 ‘근로자’를 ‘직업의 종류와 관계없이 임금을 ...
3,노동 > 근로일반,학교법인이 운영하는 대학교의 시간강사가 근로자인지,대학교에서 시간강사로 위촉한 강사들이 근로자임을 전제로 근로복지공단에서 학교법인 앞...,「근로기준법」제2조 제1항 제1호는 “‘근로자’란 직업의 종류와 관계없이 임금을 목...
4,노동 > 근로일반,자기소유 버스로 사업주의 회원 운송을 하면 근로기준법상 근로자인지,저는 제 소유 버스를 수영장 사업주의 명의로 등록하고 위 수영장에 전속되어 위 수영...,「근로기준법」상의 근로자에 해당하는지 여부에 관하여 판례는 “근로기준법상의 근로자에...


## 마지막 줄의 문구 제거

In [4]:
def remove_caution(text):
    remove_sentence = "※ 주의 : 사례에 대한 답변은 법령이나 판례 등의 변경으로 내용이 바뀔 수 있으므로 구체적인 사안에 대해서는 반드시 대한법률구조공단 상담(전화상담은 국번없이 ☎ 132) 등을 통해 다시 한 번 확인하시기 바랍니다."
    text = text.replace(remove_sentence, "").strip()
    
    return text

In [5]:
# 문구 있는 글 개수
print("문구가 있는 글 개수(전): ",len(consulting_df[consulting_df['답변'].str.contains("※ 주의 : 사례에 대한 답변은 법령이나 판례 등의 변경으로 내용이 바뀔 수 있으므로 구체적인 사안에 대해서는 반드시 대한법률구조공단 상담\(전화상담은 국번없이 ☎ 132\) 등을 통해 다시 한 번 확인하시기 바랍니다.") == True]))

# 문구 제거
consulting_df['답변'] = consulting_df.apply(lambda x: remove_caution(x.답변), axis=1)

# 문구 제거후 개수
print("문구가 있는 글 개수(후): ",len(consulting_df[consulting_df['답변'].str.contains("※ 주의 : 사례에 대한 답변은 법령이나 판례 등의 변경으로 내용이 바뀔 수 있으므로 구체적인 사안에 대해서는 반드시 대한법률구조공단 상담\(전화상담은 국번없이 ☎ 132\) 등을 통해 다시 한 번 확인하시기 바랍니다.") == True]))

문구가 있는 글 개수(전):  10034
문구가 있는 글 개수(후):  0


In [6]:
# 괄호 추출
# def extract_brackets(text):
#     p = re.compile("\([^\)]+\)")
#     brackets = p.findall(text)
    
#     brackets = [line[1:-1] for line in brackets]
    
#     return brackets

# consulting_df["괄호"] = consulting_df.apply(lambda x: extract_brackets(x.답변), axis=1)
# consulting_df

## 관련 판결문 추출

In [6]:
def extract_judgement(text):
    p = re.compile('([ㄱ-힣]*[ ]*)([0-9]{3,5}[. ]*[0-9]{1,2}[. ]*[0-9]{1,2}[.,]{0,2})?([ ]*[ㄱ-힣]*[ ]*)?([0-9]+[ ]?(가단|가소|가합|간회단|간회합|감고|감노|감도|감로|감모|감오|감초|개기|개보|개확|개회|고단|고약|고약전|고정|고합|과|구|구단|구합|국승|국지|그|나|너|노|누|느|느단|느합|다|다카|도|두|드|드단|드합|라|로|루|르|마|머|모|무|므|민상|바|버|보|부|브|사|서|성|성로|성모|성초|수흐|스|아|어|오|으|인|인라|인마|인카|자|재가단|재가소|재가합|재감고|재감노|재감도|재고단|재고약|재고정|재고합|재구|재구단|재구합|재그|재나|재너|재노|재누|재느단|재느합|재다|재도|재두|재드|재드단|재드합|재라|재루|재르|재마|재머|재무|재므|재부|재브|재비단|재비합|재스|재아|재으|재허|재후|저|저노|전고|전도|전로|전모|전오|전초|정드|정명|정브|정스|준재가단|준재가소|준재가합|준재구|준재나|준재누|준재느단|준재느합|준재드|준재드단|준재드합|준재르|준재머|준재므|준재스|즈기|즈단|즈합|차|차전|책|처|초|초기|초보|초사|초재|초적|초치|추|치고|치노|치도|치로|치모|치오|치초|카|카경|카공|카기|카기전|카단|카담|카명|카불|카소|카열|카임|카정|카조|카합|카허|카확|코|크|타기|타배|타인|타채|트|파|푸|푸초|하기|하단|하면|하합|하확|행심|허|헌가|헌나|헌다|헌라|헌마|헌바|헌사|헌아|형제|형상|호기|호명|호파|회기|회단|회합|회확|후|흐|히)[ ]?[0-9]+[,0-9 ]*)(판결|결정)?')
    judgements_match = p.finditer(text)
    judgements = [judgement.group() for judgement in judgements_match]
        
    
    return judgements

In [7]:
consulting_df['관련 판결문'] = consulting.apply(lambda x: extract_judgement(x.답변), axis=1)
consulting_df

,구분,제목,질문,답변,관련 판결문
0,노동 > 근로일반,회사분할로 퇴직금 일괄수령 후 재입사 형식 취한 경우 계속근로인지,저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서...,기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하...,"[대법원 1999. 12. 6. 선고 98다46198 판결, 2001. 9. 18..."
1,노동 > 근로일반,학습지판매위탁 상담교사도 근로기준법상의 근로자인지,저는 학습지제작·판매회사인 甲회사와 위탁업무계약을 체결한 교육상담교사입니다. 저와 ...,「근로기준법」상 근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 ...,"[대법원 1999. 2. 9. 선고 97다56235 판결, 2001. 4. 13...."
2,노동 > 근로일반,대학병원의 전공의가 근로기준법상의 근로자에 해당하는지,저는 甲대학병원에서 전공의 과정을 밟고 있는 레지던트입니다. 그런데 몇 달 전부터 ...,「근로기준법」제2조 제1항 1호에서는 ‘근로자’를 ‘직업의 종류와 관계없이 임금을 ...,"[대법원 1998. 4. 24. 선고 97다57672 판결, 2001. 3. 23..."
3,노동 > 근로일반,학교법인이 운영하는 대학교의 시간강사가 근로자인지,대학교에서 시간강사로 위촉한 강사들이 근로자임을 전제로 근로복지공단에서 학교법인 앞...,「근로기준법」제2조 제1항 제1호는 “‘근로자’란 직업의 종류와 관계없이 임금을 목...,"[대법원 1994. 12. 9. 선고 94다22859 판결, 대법원 2007. 3...."
4,노동 > 근로일반,자기소유 버스로 사업주의 회원 운송을 하면 근로기준법상 근로자인지,저는 제 소유 버스를 수영장 사업주의 명의로 등록하고 위 수영장에 전속되어 위 수영...,「근로기준법」상의 근로자에 해당하는지 여부에 관하여 판례는 “근로기준법상의 근로자에...,"[대법원 1997. 12. 26. 선고 97다17575 판결, 2001. 4. 1..."
...,...,...,...,...,...
10032,헌법 > 기타 헌법,권한쟁의심판,권한쟁의심판이 무엇인가요?,국가기관 상호간이나 지방자치단체 상호간 또는 국가기관과 지방자치단체 사이에 권한이 ...,[]
10033,헌법 > 기타 헌법,국민 개인이 대법원장을 상대로 제기한 권한쟁의심판의 가능성,대법원 패소판결을 받고 재심을 청구하였으나 기각되었습니다. 이에 저는 대법원이 법률...,"헌법재판소는, 헌법재판소가 권한쟁의심판을 청구할 수 있는 국가기관의 종류와 범위에 ...",[헌법재판소 2017. 5. 25. 자 2016헌라2]
10034,헌법 > 기타 헌법,교육감과 해당 지방자치단체 사이의 내부적 분쟁과 관련한 권한쟁의심판청구의 적법 여부,도지사가 도교육청 소속 학교들을 대상으로 계약의 적정성과 우수 식자재 위법사용 여부...,"헌법재판소는, 헌법 제111조 제1항 제4호는 지방자치단체 상호간의 권한쟁의에 관한...",[헌법재판소 2016. 6. 30. 자 2014헌라1]
10035,헌법 > 기타 헌법,정당해산결정에 대하여 재심이 허용되는지 여부,"헌법재판소의 정당해산결정에 따라 해산되었고, 소속 국회의원 5인은 국회의원직을 상실...","헌법재판소는, 정당해산심판은 원칙적으로 해당 정당에게만 그 효력이 미치며, 정당해산...",[헌법재판소 2016. 5. 26. 자 2015헌아20]


### 관련 판례없는 데이터 제거

In [9]:
count = 0

for i, d in enumerate(consulting_df['관련 판결문']):
    if len(d) == 0:
        consulting_df.loc[i, '관련 판결문'] = np.nan
        count += 1
        
print("관련 판례가 없어 제거된 데이터 개수: ", count)
consulting_have_judgement_df = consulting_df.dropna().copy().reset_index(drop=True)
consulting_have_judgement_df

관련 판례가 없어 제거된 데이터 개수:  2064


,구분,제목,질문,답변,관련 판결문
0,노동 > 근로일반,회사분할로 퇴직금 일괄수령 후 재입사 형식 취한 경우 계속근로인지,저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서...,기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하...,"[대법원 1999. 12. 6. 선고 98다46198 판결, 2001. 9. 18..."
1,노동 > 근로일반,학습지판매위탁 상담교사도 근로기준법상의 근로자인지,저는 학습지제작·판매회사인 甲회사와 위탁업무계약을 체결한 교육상담교사입니다. 저와 ...,「근로기준법」상 근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 ...,"[대법원 1999. 2. 9. 선고 97다56235 판결, 2001. 4. 13...."
2,노동 > 근로일반,대학병원의 전공의가 근로기준법상의 근로자에 해당하는지,저는 甲대학병원에서 전공의 과정을 밟고 있는 레지던트입니다. 그런데 몇 달 전부터 ...,「근로기준법」제2조 제1항 1호에서는 ‘근로자’를 ‘직업의 종류와 관계없이 임금을 ...,"[대법원 1998. 4. 24. 선고 97다57672 판결, 2001. 3. 23..."
3,노동 > 근로일반,학교법인이 운영하는 대학교의 시간강사가 근로자인지,대학교에서 시간강사로 위촉한 강사들이 근로자임을 전제로 근로복지공단에서 학교법인 앞...,「근로기준법」제2조 제1항 제1호는 “‘근로자’란 직업의 종류와 관계없이 임금을 목...,"[대법원 1994. 12. 9. 선고 94다22859 판결, 대법원 2007. 3...."
4,노동 > 근로일반,자기소유 버스로 사업주의 회원 운송을 하면 근로기준법상 근로자인지,저는 제 소유 버스를 수영장 사업주의 명의로 등록하고 위 수영장에 전속되어 위 수영...,「근로기준법」상의 근로자에 해당하는지 여부에 관하여 판례는 “근로기준법상의 근로자에...,"[대법원 1997. 12. 26. 선고 97다17575 판결, 2001. 4. 1..."
...,...,...,...,...,...
7968,헌법 > 기타 헌법,위헌으로 결정된 법률 또는 법률의 조항에 근거한 유죄의 확정판결의 재심 청구 범위,저는 2004. 8월경 간통하였다는 혐의로 기소되어 2008. 2월경 1심에서 유죄...,헌법재판소법은 제47조에서 “② 위헌으로 결정된 법률 또는 법률의 조항은 그 결정이...,[대법원 2016. 11. 10. 선고 2015모1475]
7969,헌법 > 기타 헌법,국민 개인이 대법원장을 상대로 제기한 권한쟁의심판의 가능성,대법원 패소판결을 받고 재심을 청구하였으나 기각되었습니다. 이에 저는 대법원이 법률...,"헌법재판소는, 헌법재판소가 권한쟁의심판을 청구할 수 있는 국가기관의 종류와 범위에 ...",[헌법재판소 2017. 5. 25. 자 2016헌라2]
7970,헌법 > 기타 헌법,교육감과 해당 지방자치단체 사이의 내부적 분쟁과 관련한 권한쟁의심판청구의 적법 여부,도지사가 도교육청 소속 학교들을 대상으로 계약의 적정성과 우수 식자재 위법사용 여부...,"헌법재판소는, 헌법 제111조 제1항 제4호는 지방자치단체 상호간의 권한쟁의에 관한...",[헌법재판소 2016. 6. 30. 자 2014헌라1]
7971,헌법 > 기타 헌법,정당해산결정에 대하여 재심이 허용되는지 여부,"헌법재판소의 정당해산결정에 따라 해산되었고, 소속 국회의원 5인은 국회의원직을 상실...","헌법재판소는, 정당해산심판은 원칙적으로 해당 정당에게만 그 효력이 미치며, 정당해산...",[헌법재판소 2016. 5. 26. 자 2015헌아20]


In [11]:
for i, d in enumerate(consulting_have_judgement_df['관련 판결문']):
    print(i, d)

0 ['대법원 1999. 12. 6. 선고 98다46198 판결', ' 2001. 9. 18. 선고 2000다60630 판결', '대법원 1992. 7. 14. 선고 91다40276 판결', ' 2005. 2. 25. 선고 2004다34790 판결']
1 ['대법원 1999. 2. 9. 선고 97다56235 판결', ' 2001. 4. 13. 선고 2000도4901 판결', ' 2002. 7. 26. 선고 2000다27671 판결', '대법원 1996. 4. 26. 선고 95다20348 판결', ' 2005. 11. 24. 선고 2005다39136 판결']
2 ['대법원 1998. 4. 24. 선고 97다57672 판결', ' 2001. 3. 23. 선고 2000다39513 판결']
3 ['대법원 1994. 12. 9. 선고 94다22859 판결', '대법원 2007. 3. 29. 선고 2005두13018, 13025 판결', '대법원 2006. 12. 7. 선고 2004다29736 판결']
4 ['대법원 1997. 12. 26. 선고 97다17575 판결', ' 2001. 4. 13. 선고 2000도4901 판결', ' 2002. 7. 26. 선고 2000다27671 판결', '대법원 2000. 1. 18. 선고 99다48986 판결']
5 ['서울지법 1996. 9. 10. 선고 96가단90373 판결', '대법원 1979. 3. 27. 선고 78다163 판결', '대법원 1987. 4. 14. 선고 87도 153 판결', '대법원 1997. 11. 28. 선고 97다28971 판결', ' 2000. 3. 14. 선고 99도1243 판결']
6 ['대법원 1995. 12. 22. 선고 95누2050 판결', ' 1997. 10. 10. 선고 97누10352 판결']
7 ['대법원 1992. 12. 22. 선고 92다28228 판결', '대법원 1992. 5. 12. 선고 91누11490 판결', ' 2005. 5. 27. 선고 2005두524 판결']
8 

In [13]:
p = re.compile('(?P<court>[ㄱ-힣]*[ ]*)(?P<date>[0-9]{3,5}[. ]*[0-9]{1,2}[. ]*[0-9]{1,2}[.,]{0,2})?([ ]*[ㄱ-힣]*[ ]*)?(?P<case_number>[0-9]+[ ]?(가단|가소|가합|간회단|간회합|감고|감노|감도|감로|감모|감오|감초|개기|개보|개확|개회|고단|고약|고약전|고정|고합|과|구|구단|구합|국승|국지|그|나|너|노|누|느|느단|느합|다|다카|도|두|드|드단|드합|라|로|루|르|마|머|모|무|므|민상|바|버|보|부|브|사|서|성|성로|성모|성초|수흐|스|아|어|오|으|인|인라|인마|인카|자|재가단|재가소|재가합|재감고|재감노|재감도|재고단|재고약|재고정|재고합|재구|재구단|재구합|재그|재나|재너|재노|재누|재느단|재느합|재다|재도|재두|재드|재드단|재드합|재라|재루|재르|재마|재머|재무|재므|재부|재브|재비단|재비합|재스|재아|재으|재허|재후|저|저노|전고|전도|전로|전모|전오|전초|정드|정명|정브|정스|준재가단|준재가소|준재가합|준재구|준재나|준재누|준재느단|준재느합|준재드|준재드단|준재드합|준재르|준재머|준재므|준재스|즈기|즈단|즈합|차|차전|책|처|초|초기|초보|초사|초재|초적|초치|추|치고|치노|치도|치로|치모|치오|치초|카|카경|카공|카기|카기전|카단|카담|카명|카불|카소|카열|카임|카정|카조|카합|카허|카확|코|크|타기|타배|타인|타채|트|파|푸|푸초|하기|하단|하면|하합|하확|행심|허|헌가|헌나|헌다|헌라|헌마|헌바|헌사|헌아|형제|형상|호기|호명|호파|회기|회단|회합|회확|후|흐|히)[ ]?[0-9]+[,0-9 ]*)(판결|결정)?')
def add_case_number(judgements):
    return [p.match(judgement).group('case_number') for judgement in judgements]

consulting_have_judgement_df['관련 사건번호'] = consulting_have_judgement_df.apply(lambda x: add_case_number(x["관련 판결문"]), axis=1)
consulting_have_judgement_df

,구분,제목,질문,답변,관련 판결문,관련 사건번호
0,노동 > 근로일반,회사분할로 퇴직금 일괄수령 후 재입사 형식 취한 경우 계속근로인지,저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서...,기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하...,"[대법원 1999. 12. 6. 선고 98다46198 판결, 2001. 9. 18...","[98다46198 , 2000다60630 , 91다40276 , 2004다34790 ]"
1,노동 > 근로일반,학습지판매위탁 상담교사도 근로기준법상의 근로자인지,저는 학습지제작·판매회사인 甲회사와 위탁업무계약을 체결한 교육상담교사입니다. 저와 ...,「근로기준법」상 근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 ...,"[대법원 1999. 2. 9. 선고 97다56235 판결, 2001. 4. 13....","[97다56235 , 2000도4901 , 2000다27671 , 95다20348 ..."
2,노동 > 근로일반,대학병원의 전공의가 근로기준법상의 근로자에 해당하는지,저는 甲대학병원에서 전공의 과정을 밟고 있는 레지던트입니다. 그런데 몇 달 전부터 ...,「근로기준법」제2조 제1항 1호에서는 ‘근로자’를 ‘직업의 종류와 관계없이 임금을 ...,"[대법원 1998. 4. 24. 선고 97다57672 판결, 2001. 3. 23...","[97다57672 , 2000다39513 ]"
3,노동 > 근로일반,학교법인이 운영하는 대학교의 시간강사가 근로자인지,대학교에서 시간강사로 위촉한 강사들이 근로자임을 전제로 근로복지공단에서 학교법인 앞...,「근로기준법」제2조 제1항 제1호는 “‘근로자’란 직업의 종류와 관계없이 임금을 목...,"[대법원 1994. 12. 9. 선고 94다22859 판결, 대법원 2007. 3....","[94다22859 , 2005두13018, 13025 , 2004다29736 ]"
4,노동 > 근로일반,자기소유 버스로 사업주의 회원 운송을 하면 근로기준법상 근로자인지,저는 제 소유 버스를 수영장 사업주의 명의로 등록하고 위 수영장에 전속되어 위 수영...,「근로기준법」상의 근로자에 해당하는지 여부에 관하여 판례는 “근로기준법상의 근로자에...,"[대법원 1997. 12. 26. 선고 97다17575 판결, 2001. 4. 1...","[97다17575 , 2000도4901 , 2000다27671 , 99다48986 ]"
...,...,...,...,...,...,...
7968,헌법 > 기타 헌법,위헌으로 결정된 법률 또는 법률의 조항에 근거한 유죄의 확정판결의 재심 청구 범위,저는 2004. 8월경 간통하였다는 혐의로 기소되어 2008. 2월경 1심에서 유죄...,헌법재판소법은 제47조에서 “② 위헌으로 결정된 법률 또는 법률의 조항은 그 결정이...,[대법원 2016. 11. 10. 선고 2015모1475],[2015모1475]
7969,헌법 > 기타 헌법,국민 개인이 대법원장을 상대로 제기한 권한쟁의심판의 가능성,대법원 패소판결을 받고 재심을 청구하였으나 기각되었습니다. 이에 저는 대법원이 법률...,"헌법재판소는, 헌법재판소가 권한쟁의심판을 청구할 수 있는 국가기관의 종류와 범위에 ...",[헌법재판소 2017. 5. 25. 자 2016헌라2],[2016헌라2]
7970,헌법 > 기타 헌법,교육감과 해당 지방자치단체 사이의 내부적 분쟁과 관련한 권한쟁의심판청구의 적법 여부,도지사가 도교육청 소속 학교들을 대상으로 계약의 적정성과 우수 식자재 위법사용 여부...,"헌법재판소는, 헌법 제111조 제1항 제4호는 지방자치단체 상호간의 권한쟁의에 관한...",[헌법재판소 2016. 6. 30. 자 2014헌라1],[2014헌라1]
7971,헌법 > 기타 헌법,정당해산결정에 대하여 재심이 허용되는지 여부,"헌법재판소의 정당해산결정에 따라 해산되었고, 소속 국회의원 5인은 국회의원직을 상실...","헌법재판소는, 정당해산심판은 원칙적으로 해당 정당에게만 그 효력이 미치며, 정당해산...",[헌법재판소 2016. 5. 26. 자 2015헌아20],[2015헌아20]
